In [1]:
# !pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install detectron2==0.6 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html
# !pip install setuptools==59.5.0

In [7]:
import detectron2
import detectron2.data.transforms as T
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader, build_detection_test_loader
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.logger import setup_logger

setup_logger()

import numpy as np
import os
import random
import torch, torchvision

In [3]:
DETECTRON_2_BASE_PATH = "/tf/datasets/final-dataset/detectron2-dataset"

DETECTRON_2_DATASET_PATHS = {
    "train": os.path.join(DETECTRON_2_BASE_PATH, "train"),
    "val": os.path.join(DETECTRON_2_BASE_PATH, "val"),
    "test": os.path.join(DETECTRON_2_BASE_PATH, "test"),
}

for subset in DETECTRON_2_DATASET_PATHS.keys():
    register_coco_instances(
        name=f"custom_dataset_{subset}",
        metadata={},
        json_file=os.path.join(
            DETECTRON_2_DATASET_PATHS[subset], "_annotations.coco.json"
        ),
        image_root=DETECTRON_2_DATASET_PATHS[subset],
    )

In [4]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(
            cfg,
            is_train=True,
            augmentations=[
                T.Resize((400, 400)),
                T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                T.RandomSaturation(intensity_min=0.8, intensity_max=1.2),
                T.RandomBrightness(intensity_min=0.8, intensity_max=1.2),
                T.RandomContrast(intensity_min=0.8, intensity_max=1.2),
            ],
        )
        return build_detection_train_loader(cfg, mapper=mapper)

    @classmethod
    def build_test_loader(cls, cfg, name="test"):
        mapper = DatasetMapper(
            cfg,
            is_train=False,
            augmentations=[T.Resize((400, 400))],
        )
        return build_detection_test_loader(cfg, name, mapper=mapper)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):

        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True) # arreglar esto
            output_folder = "coco_eval"

        return COCOEvaluator(dataset_name, ("bbox",), False, output_folder)

In [5]:
RUNS_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/runs"
TRAIN_PROJECT_PATH = os.path.join(RUNS_PATH, "train")

experiment = 'detectron2-rcnn-2000-epochs'
experimentPath =  os.path.join(TRAIN_PROJECT_PATH, experiment)


CONFIG_FILE = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE))
cfg.DATASETS.TRAIN = ("custom_dataset_train",)
cfg.DATASETS.TEST = ("custom_dataset_val",)
cfg.OUTPUT_DIR = experimentPath


cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE)
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001

# cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.TEST.EVAL_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [6]:
torch.cuda.empty_cache() 

trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[10/08 06:24:47 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tensorboard/compat/__init__.py", line 42, in tf
    from tensorboard.compat import notf  # noqa: F401
ImportError: cannot import name 'notf' from 'tensorboard.compat' (/usr/local/lib/python3.8/dist-packages/tensorboard/compat/__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 1934, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 1936, in showtraceback
    stb = self.InteractiveTB.structured_traceback(etype,
  File "/usr/local/lib/python3.8/dist-packages/IPython

[10/08 06:24:48 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.8/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.8/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/08 06:25:13 d2.utils.events]:  eta: 0:40:32  iter: 19  total_loss: 0.8043  loss_cls: 0.6663  loss_box_reg: 0.04169  loss_rpn_cls: 0.08693  loss_rpn_loc: 0.007043  time: 1.2210  data_time: 0.1905  lr: 1.931e-06  max_mem: 4667M
[10/08 06:25:38 d2.utils.events]:  eta: 0:39:38  iter: 39  total_loss: 0.7714  loss_cls: 0.6346  loss_box_reg: 0.04371  loss_rpn_cls: 0.08704  loss_rpn_loc: 0.007734  time: 1.2218  data_time: 0.1668  lr: 2.911e-06  max_mem: 4667M
[10/08 06:26:01 d2.utils.events]:  eta: 0:38:30  iter: 59  total_loss: 0.7228  loss_cls: 0.5747  loss_box_reg: 0.04906  loss_rpn_cls: 0.08758  loss_rpn_loc: 0.006913  time: 1.2042  data_time: 0.1100  lr: 3.891e-06  max_mem: 4667M
[10/08 06:26:25 d2.utils.events]:  eta: 0:38:11  iter: 79  total_loss: 0.6348  loss_cls: 0.4958  loss_box_reg: 0.04592  loss_rpn_cls: 0.09372  loss_rpn_loc: 0.00831  time: 1.2075  data_time: 0.1604  lr: 4.871e-06  max_mem: 4667M
[10/08 06:26:50 d2.utils.events]:  eta: 0:37:49  iter: 99  total_loss: 0.5617  lo

In [ ]:
# run tensorboard
# !tensorboard --logdir=/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train --host 0.0.0.0

In [8]:
#test evaluation
# from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("custom_dataset_test", cfg, False, output_dir="./output/")
mapper = DatasetMapper(cfg, is_train=False, augmentations=[T.Resize((400, 400))])
test_loader = build_detection_test_loader(cfg, "custom_dataset_test", mapper=mapper)
# test_loader = build_detection_test_loader(cfg, "custom_dataset_test")
inference_on_dataset(trainer.model, test_loader, evaluator) 

WARNING [10/08 11:14:05 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[10/08 11:14:05 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [Resize(shape=(400, 400))]
[10/08 11:14:05 d2.data.datasets.coco]: Loaded 718 images in COCO format from /tf/datasets/final-dataset/detectron2-dataset/test/_annotations.coco.json
[10/08 11:14:05 d2.data.build]: Distribution of instances among all 1 categories:
|   category    | #instances   |
|:-------------:|:-------------|
| license_plate | 718          |
|               |              |
[10/08 11:14:05 d2.data.common]: Serializing 718 elements to byte tensors and concatenating them all ...
[10/08 11:14:05 d2.data.common]: Serialized dataset takes 0.18 MiB
[10/08 11:14:05 d2.evaluation.evaluator]: Start inference on 718 batches
[10/08 11:14:06 d2.evaluation.evaluator]: Inference done 11/718. Dataloading: 0.0008 s/iter. Inference

OrderedDict([('bbox',
              {'AP': 40.35250476360136,
               'AP50': 79.44063866583815,
               'AP75': 30.321568188808286,
               'APs': 16.68424210689909,
               'APm': 41.4377444524639,
               'APl': 44.972794621546285})])

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
test_metadata

namespace(name='my_dataset_test')